<a href="https://colab.research.google.com/github/ytg000629/Dacon_Dobae/blob/main/modeling/ResNet50.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import os
import pandas as pd
import numpy as np
import seaborn as sns
from PIL import Image
import random
import cv2

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [4]:
#전처리 파라미터 설정

from keras.preprocessing.image import ImageDataGenerator

data_with_augmentation = ImageDataGenerator(width_shift_range = 0.1,
                                            height_shift_range = 0.1,
                                            horizontal_flip = True,
                                            vertical_flip = True,
                                            fill_mode = 'nearest',
                                            shear_range = 0.1,
                                            rescale = 1/255.0
                                            )

data_with_no_augmentation = ImageDataGenerator(rescale = 1/255.0)

In [5]:
train = data_with_augmentation.flow_from_directory(directory = '/content/drive/MyDrive/dacon/data/data_split/train',
                                                                  target_size = (224, 224), # ResNet50 input shape = (224, 224, 3)
                                                                  batch_size = 32,
                                                                  class_mode = 'categorical')

Found 2410 images belonging to 19 classes.


In [6]:
val = data_with_no_augmentation.flow_from_directory(directory = '/content/drive/MyDrive/dacon/data/data_split/val',
                                                                  target_size = (224, 224), # ResNet50 input shape = (224, 224, 3)
                                                                  batch_size = 32,
                                                                  class_mode = 'categorical')

Found 1047 images belonging to 19 classes.


In [7]:
train.class_indices

{'가구수정': 0,
 '걸레받이수정': 1,
 '곰팡이': 2,
 '꼬임': 3,
 '녹오염': 4,
 '들뜸': 5,
 '면불량': 6,
 '몰딩수정': 7,
 '반점': 8,
 '석고수정': 9,
 '오염': 10,
 '오타공': 11,
 '울음': 12,
 '이음부불량': 13,
 '창틀,문틀수정': 14,
 '터짐': 15,
 '틈새과다': 16,
 '피스': 17,
 '훼손': 18}

In [8]:
val.class_indices

{'가구수정': 0,
 '걸레받이수정': 1,
 '곰팡이': 2,
 '꼬임': 3,
 '녹오염': 4,
 '들뜸': 5,
 '면불량': 6,
 '몰딩수정': 7,
 '반점': 8,
 '석고수정': 9,
 '오염': 10,
 '오타공': 11,
 '울음': 12,
 '이음부불량': 13,
 '창틀,문틀수정': 14,
 '터짐': 15,
 '틈새과다': 16,
 '피스': 17,
 '훼손': 18}

In [9]:
import keras
from keras.applications import ResNet50
from keras.layers import Dropout, Dense, GlobalAveragePooling2D
from keras.models import Model
from keras.layers import GlobalMaxPooling2D
import tensorflow as tf

base_model = ResNet50(include_top = False, weights = 'imagenet', input_shape = (224, 224, 3))
base_model.summary()

Model: "resnet50"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_1 (InputLayer)           [(None, 224, 224, 3  0           []                               
                                )]                                                                
                                                                                                  
 conv1_pad (ZeroPadding2D)      (None, 230, 230, 3)  0           ['input_1[0][0]']                
                                                                                                  
 conv1_conv (Conv2D)            (None, 112, 112, 64  9472        ['conv1_pad[0][0]']              
                                )                                                                 
                                                                                           

In [10]:
base_model.trainable = False

In [ ]:
from tensorflow.keras.applications.resnet50 import preprocess_input

In [11]:
from tensorflow import keras
from keras.applications import ResNet50

model = keras.Sequential()
model.add(keras.Input(shape = (224, 224, 3)))
model.add(base_model)
model.add(keras.layers.Flatten())
model.add(keras.layers.Dense(256, activation='relu'))
model.add(Dropout(0.2))
model.add(keras.layers.Dense(19, activation = 'softmax'))
model.summary()


Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 resnet50 (Functional)       (None, 7, 7, 2048)        23587712  
                                                                 
 flatten (Flatten)           (None, 100352)            0         
                                                                 
 dense (Dense)               (None, 256)               25690368  
                                                                 
 dropout (Dropout)           (None, 256)               0         
                                                                 
 dense_1 (Dense)             (None, 19)                4883      
                                                                 
Total params: 49,282,963
Trainable params: 25,695,251
Non-trainable params: 23,587,712
_________________________________________________________________


In [ ]:
#!pip install focal_loss

In [ ]:
#!pip install git+https://github.com/artemmavrin/focal-loss.git

In [13]:
#from focal_loss import SparseCategoricalFocalLoss

In [ ]:
#def focal_loss(gamma=2., alpha=4.):

    #gamma = float(gamma)
    #alpha = float(alpha)

    #def focal_loss_fixed(y_true, y_pred):
        #epsilon = 1.e-9
        #y_true = tf.convert_to_tensor(y_true, tf.float32)
        #y_pred = tf.convert_to_tensor(y_pred, tf.float32)

        #model_out = tf.add(y_pred, epsilon)
        #ce = tf.multiply(y_true, -tf.math.log(model_out))
        #weight = tf.multiply(y_true, tf.pow(tf.subtract(1., model_out), gamma))
        #fl = tf.multiply(alpha, tf.multiply(weight, ce))
        #reduced_fl = tf.reduce_max(fl, axis=1)
        #return tf.reduce_mean(reduced_fl)
    #return focal_loss_fixed

In [34]:
#from keras import backend as K
#def focal_loss(y_true, y_pred):
    #gamma = 2.0,
    #alpha = 0.25,
    #pt_1 = tf.where(tf.equal(y_true, 1), y_pred, tf.ones_like(y_pred)),
    #pt_0 = tf.where(tf.equal(y_true, 0), y_pred, tf.zeros_like(y_pred))
    #return -K.sum(alpha * K.pow(1. - pt_1, gamma) * K.log(pt_1))-K.sum((1-alpha) * K.pow( pt_0, gamma) * K.log(1. - pt_0))

In [14]:
!pip install tensorflow-addons==0.16.1

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [15]:
import tensorflow_addons.metrics

/usr/local/lib/python3.10/dist-packages/tensorflow_addons/utils/ensure_tf_install.py:53: UserWarning: Tensorflow Addons supports using Python ops for all Tensorflow versions above or equal to 2.6.0 and strictly below 2.9.0 (nightly versions are not supported). 
 The versions of TensorFlow you are currently using is 2.12.0 and is not supported. 
Some things might work, some things might not.
If you were to encounter a bug, do not file an issue.
If you want to make sure you're using a tested and supported configuration, either change the TensorFlow version or the TensorFlow Addons's version. 
You can find the compatibility matrix in TensorFlow Addon's readme:
https://github.com/tensorflow/addons
  warnings.warn(


In [26]:
w_f1 = tensorflow_addons.metrics.F1Score(num_classes=19, average = 'weighted')

In [39]:
from keras.optimizers import Adam
model.compile(optimizer = Adam(learning_rate = 0.001),
              loss = 'categorical_crossentropy',
              #loss = [focal_loss],
              #loss = CategoricalFocalLoss(gamma = 2),
              #loss = focal_loss(gamma = 2, alpha = 4),
              metrics = ['accuracy', w_f1])

In [36]:
from keras.callbacks import ReduceLROnPlateau, EarlyStopping, ModelCheckpoint

#rlr_cb = ReduceLROnPlateau(monitor = 'val_f1_score', factor = 0.1, patience = 5, mode = 'min', verbose = 1)
ely_cb = EarlyStopping(monitor = 'val_f1_score', patience = 10, mode = 'min', verbose = 1)
mc_cb = ModelCheckpoint('model.h5', verbose=1, save_best_only=True, save_weights_only=True)


In [18]:
model.input_shape

(None, 224, 224, 3)

In [40]:
from sklearn.utils.class_weight import compute_class_weight

# Get the number of samples in each class in the training set
train_class_counts = train.classes
num_classes = len(np.unique(train_class_counts))

# Compute class weights based on the training set
class_weights = compute_class_weight(class_weight='balanced',
                                     classes=np.unique(train_class_counts),
                                     y=train_class_counts)

class_weights = dict(zip(np.unique(train_class_counts), class_weights))
class_weights

#class_weights 적용하니까 fit했을 때 결과가 이상함

{0: 15.855263157894736,
 1: 0.5927201180521396,
 2: 1.255862428348098,
 3: 0.8628714643752238,
 4: 14.09356725146199,
 5: 3.4281650071123755,
 6: 1.8382913806254768,
 7: 1.3938692886061308,
 8: 63.421052631578945,
 9: 3.252361673414305,
 10: 0.3049089068825911,
 11: 1.2812333864965444,
 12: 8.456140350877194,
 13: 11.5311004784689,
 14: 7.046783625730995,
 15: 1.1224965067536097,
 16: 42.280701754385966,
 17: 3.6240601503759398,
 18: 0.12903571237350753}

In [ ]:
history = model.fit(train, epochs = 50, validation_data = val, callbacks = [ely_cb, mc_cb])

In [ ]:
from matplotlib import pyplot as plt

plt.plot(history.history['loss'], 'r', label = 'Train Loss')
plt.plot(history.history['val_loss'], 'b', label = 'Val Loss')
plt.title('Training and Validation Loss')
plt.legend()

plt.show()

plt.plot(history.history['accuracy'], 'r', label = 'Train Accuracy')
plt.plot(history.history['val_accuracy'], 'b', label = 'Val Accuracy')
plt.title('Training and Validation Accuracy')
plt.legend()

plt.show()

plt.plot(history.history['f1_score'], 'r', label = 'Train F1 Score')
plt.plot(history.history['val_f1_score'], 'b', label = 'Val F1 Score')
plt.title('Training and Validation F1 Score')
plt.legend()

plt.show()